### Day 13: Transparent Origami

In [9]:
sample = """
6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0

fold along y=7
fold along x=5""".strip().split('\n')

In [99]:
import numpy as np

def parse_transparent_paper(lines):
    dots, folds = [], []
    for l in lines:
        if l.startswith('fold'):
            axis, pos = l.replace('fold along ', '').split('=')
            folds.append((axis, int(pos)))
        elif l:
            dots.append(tuple(map(int, l.split(','))))
    return dots, folds

class TransparentPaper:
    
    def __init__(self, dots):
        self.size_x = max(map(lambda pos: pos[0], dots)) + 1
        self.size_y = max(map(lambda pos: pos[1], dots)) + 1
        self.pattern = np.zeros((self.size_y, self.size_x), dtype=int)
        for x,y in dots:
            self.pattern[y,x] = 1
    
    def __repr__(self):
        return '\n'.join(''.join('.' if x == 0 else '#' if x == 1 else '' for x in row) for row in self.pattern)
    
    def fold_up(self, pos):
        fold_size = max(pos, self.size_y - pos - 1)
        to_fold = self._pad_up(np.flipud(self.pattern[pos+1:]), fold_size)
        folded = self._pad_up(self.pattern[:pos], fold_size)
        folded[np.where(to_fold == 1)] = 1
        self.pattern = folded
        self.size_x = self.pattern.shape[1]
        self.size_y = self.pattern.shape[0]
        
    def fold_left(self, pos):
        fold_size = max(pos, self.size_x - pos - 1)
        to_fold = self._pad_left(np.fliplr(self.pattern[:,pos+1:]), fold_size)
        folded = self._pad_left(self.pattern[:,:pos], fold_size)
        folded[np.where(to_fold == 1)] = 1
        self.pattern = folded
        self.size_x = self.pattern.shape[1]
        self.size_y = self.pattern.shape[0]
        
    def do_folds(self, folds):
        for fold, pos in folds:
            (self.fold_left if fold == 'x' else self.fold_up)(pos)
            
    def get_visible_dots(self):
        return self.pattern.sum()
    
    @staticmethod
    def _pad_left(arr, target_width):
        pad_width = target_width - arr.shape[1]
        return arr if not pad_width else np.pad(arr, ((0, 0), (pad_width, 0)))
    
    @staticmethod
    def _pad_up(arr, target_width):
        pad_width = target_width - arr.shape[0]
        return arr if not pad_width else np.pad(arr, ((pad_width, 0), (0, 0)))

In [106]:
dots, folds = parse_transparent_paper(sample)
print(f'# dots = {len(dots)} (e.g. {dots[0]})\n# folds = {len(folds)} (e.g. {folds[0]})')

tp = TransparentPaper(dots)
tp.do_folds([folds[0]])
assert tp.get_visible_dots() == 17
tp.do_folds(folds[1:])
print('');display(tp)
tp.fold_left(3)
print('');display(tp)
tp.fold_up(1)
print('');tp

# dots = 18 (e.g. (6, 10))
# folds = 2 (e.g. ('y', 7))



#####
#...#
#...#
#...#
#####
.....
.....

###
#.#
#.#
#.#
###
...
...

...
...
###
#.#
###

In [130]:
# Part 1
dots, folds = parse_transparent_paper(map(str.strip, open('data/input_day13.txt').readlines()))
print(f'# dots = {len(dots)} (e.g. {dots[0]})\n# folds = {len(folds)} (e.g. {folds[0]})')

tp = TransparentPaper(dots)
tp.do_folds([folds[0]])
tp.get_visible_dots()

# dots = 1004 (e.g. (1284, 229))
# folds = 12 (e.g. ('x', 655))


847

In [131]:
# Part 2
tp.do_folds(folds[1:])
print(tp.pattern.shape)
tp  # BCZRCEAB

(6, 40)


###...##..####.###...##..####..##..###..
#..#.#..#....#.#..#.#..#.#....#..#.#..#.
###..#......#..#..#.#....###..#..#.###..
#..#.#.....#...###..#....#....####.#..#.
#..#.#..#.#....#.#..#..#.#....#..#.#..#.
###...##..####.#..#..##..####.#..#.###..

### Day 14: Extended Polymerization

In [1]:
sample = """
NNCB

CH -> B
HH -> N
CB -> H
NH -> C
HB -> C
HC -> B
HN -> C
NN -> C
BH -> H
NC -> B
NB -> B
BN -> B
BB -> N
BC -> B
CC -> N
CN -> C""".strip().split('\n')

In [52]:
from itertools import tee
from collections import Counter, defaultdict

def parse_instructions(lines):
    return lines[0].strip(), {pair: new for pair, new in map(lambda s: s.strip().split(' -> '), lines[2:])}

def polymerisation_step(polymer_generator, pair_insertions):
    def new_polymer():
        first_elements, second_elements = tee(polymer_generator)
        next(second_elements, None)
        for el1, el2 in zip(first_elements, second_elements):
            el_pair = el1 + el2
            yield(el1)
            if el_pair in pair_insertions:
                yield(pair_insertions[el_pair])
        yield(el2)
    return new_polymer()

def polymerisation(polymer_template, pair_insertions, n_steps=1, return_calc=False):
    res = polymer_template[:]
    
    for i in range(n_steps):
        res = polymerisation_step(res, pair_insertions)
        
    if not return_calc:
        return ''.join(res)  # Polymerisation string
    
    c = Counter(res)
    return c.most_common()[0][1] - c.most_common()[-1][1]  # Calc: count most common - count least common

polymer_template, pair_insertions = parse_instructions(sample)
polymer_template, pair_insertions
assert ''.join(polymerisation_step(polymer_template, pair_insertions)) == 'NCNBCHB'
assert polymerisation(polymer_template, pair_insertions, 1) == 'NCNBCHB'
assert polymerisation(polymer_template, pair_insertions, 2) == 'NBCCNBBBCBHCB'
assert polymerisation(polymer_template, pair_insertions, 3) == 'NBBBCNCCNBBNBNBBCHBHHBCHB'
assert polymerisation(polymer_template, pair_insertions, 4) == 'NBBNBNBBCCNBCNCCNBBNBBNBBBNBBNBBCBHCBHHNHCBBCBHCB'
assert len(polymerisation(polymer_template, pair_insertions, 10)) == 3073
assert polymerisation(polymer_template, pair_insertions, 10, return_calc=True) == 1588

In [16]:
polymerisation(*parse_instructions(open('data/input_day14.txt').readlines()), 10, return_calc=True)

2975

In [59]:
def polymerisation_step_fast(current_polymer_pairs, pair_insertions):
    new_polymer_pairs = current_polymer_pairs.copy()
    for pair, inserted_el in pair_insertions.items():
        pair_count = current_polymer_pairs.get(pair, 0)
        if pair_count:
            new_polymer_pairs[pair] -= pair_count
            new_polymer_pairs[pair[0] + inserted_el] += pair_count
            new_polymer_pairs[inserted_el + pair[1]] += pair_count
    return new_polymer_pairs

def polymerisation_fast(polymer_template, pair_insertions, n_steps=1, return_calc=False):
    # Generate and count pairs
    el_pairs = defaultdict(int, Counter(map(''.join, zip('-' + polymer_template, polymer_template + '-'))))
    for i in range(n_steps):
        el_pairs = polymerisation_step_fast(el_pairs, pair_insertions)
    
    # Convert to element counts
    el_count = defaultdict(float)
    for pair, count in el_pairs.items():
        for el in pair:
            if el != '-':
                el_count[el] += count / 2

    return int(max(el_count.values()) - min(el_count.values()))

assert polymerisation_fast(polymer_template, pair_insertions, 10) == 1588
assert polymerisation_fast(polymer_template, pair_insertions, 40) == 2188189693529

In [60]:
polymerisation_fast(*parse_instructions(open('data/input_day14.txt').readlines()), 40, return_calc=True)

3015383850689

### Day 15: Chiton

In [9]:
sample = """
1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581
""".strip().split('\n')
sample = list(map(lambda l: list(map(int, l)), sample))
sample[0]

[1, 1, 6, 3, 7, 5, 1, 7, 4, 2]

In [10]:
# Part 1

def parse_map(file_name):
    return list(map(lambda l: list(map(int, l.strip())), open(file_name).readlines()))

def min_path_cost(cell_cost, diagonal=False):
    row, col = len(cell_cost) - 1, len(cell_cost[0]) - 1
    cost = [row.copy() for row in cell_cost]
    
    for i in range(row):
        cost[i+1][0] += cost[i][0]
        
    for j in range(col):
        cost[0][j+1] += cost[0][j]
        
    for i in range(row):
        for j in range(col):
            neighbours = [cost[i][j+1], cost[i+1][j]]
            min_neighbour_cost = min(*neighbours)
            cost[i+1][j+1] += (min(cost[i][j], min_neighbour_cost)
                               if diagonal else
                               min_neighbour_cost)
    return cost[row][col] - cost[0][0]

cost = [[1, 2, 3], 
        [4, 8, 2], 
        [1, 5, 3]]
assert min_path_cost(cost, diagonal=True) == 7
assert min_path_cost(cost) == 10
assert min_path_cost(sample) == 40

In [11]:
min_path_cost(parse_map('data/input_day15.txt'))

745

In [12]:
# Part 2

def to_full_map(small_map):
    return [[x + i + j if x + i + j < 10 else x + i + j - 9 
             for j in range(5)
             for x in row ]
            for i in range(5)
            for row in small_map]
    
assert to_full_map([[8]]) == [[8, 9, 1, 2, 3],
                              [9, 1, 2, 3, 4],
                              [1, 2, 3, 4, 5],
                              [2, 3, 4, 5, 6],
                              [3, 4, 5, 6, 7]]
assert to_full_map(sample) == parse_map('data/sample_part2_day15.txt')
assert min_path_cost(to_full_map(sample)) == 315

In [13]:
min_path_cost(to_full_map(parse_map('data/input_day15.txt')))  # Wrong answer :(

3012

In [14]:
# An example that shows the issue with the first algorithm: moves up and left aren't taken into account
cost2 = [[1,9,9,9,9],
         [1,9,1,1,1],
         [1,1,1,9,1]]
assert min_path_cost(cost2) == 8

AssertionError: 

In [15]:
import networkx as nx
from matplotlib import pyplot as plt
from networkx.algorithms.shortest_paths.weighted import bidirectional_dijkstra

def create_grid_graph(grid):
    G = nx.grid_2d_graph(len(grid), len(grid[0]))
    for node in G.nodes():
        G.nodes[node]['weight'] = grid[node[0]][node[1]]
    return G
    
def min_path_cost_2(cell_cost, draw_graph=False):
    G = create_grid_graph(cell_cost)
    source, target = (0,0), (len(cell_cost)-1, len(cell_cost[0])-1)
    
    def get_edge_weight(u, v, edge_attr):
        return G.nodes[v]['weight']
    _, path = bidirectional_dijkstra(G, source, target, weight=get_edge_weight)
    
    path_cost = sum(cell_cost[i][j] for i,j in path[1:])
    return path_cost

assert min_path_cost_2(cost) == 10
assert min_path_cost_2(cost2) == 8
assert min_path_cost_2(sample) == 40

In [16]:
min_path_cost_2(to_full_map(parse_map('data/input_day15.txt')))

3002